In [ ]:
!pip install pyts

# Import library and data

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(20.27,10)})
import tensorflow as tf

In [ ]:
from google.colab import drive

In [ ]:
#import data
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_csv('gdrive/My Drive/MBKM/gladius_v3.csv')

In [ ]:
df.reset_index(inplace = True)

In [ ]:
def late(x):
  if x == 0:
    return 'UNPAID'
  elif (x >= 1) & (x<21):
    return 'PAID'
  else:
    return 'LATE PAID'

for i in range(11, 0, -1):
  df[f'month_{i}_late'] = df['billing_{}_paymentDay'.format(i)].apply(lambda x : late(x))

In [ ]:
df

index  Unnamed: 0                       _id  billing_11_amountTotal  \
0          0           0  617fc8959da5e900114047e8                344000.0   
1          1           1  617fc8959da5e900114047e9                327500.0   
2          2           2  617fc8969da5e900114047ea                145145.0   
3          3           3  617fc8969da5e900114047eb                209660.0   
4          4           4  617fc8969da5e900114047ec                275000.0   
...      ...         ...                       ...                     ...   
95531  95531       99992  618053dd9da5e9001141ce81                     0.0   
95532  95532       99993  618053dd9da5e9001141ce82                     0.0   
95533  95533       99994  618053de9da5e9001141ce83                     0.0   
95534  95534       99995  618053e19da5e9001141ce84                     0.0   
95535  95535       99996  618053ff9da5e9001141ce85                     0.0   

                        billing_11_channel billing_11_currency  \
0          FINNET BANK - BANK CENTRAL ASIA                 IDR   
1                 FINNET SUBCA - TOKOPEDIA                 IDR   
2               FINNET BANK - BANK MANDIRI                 IDR   
3      FINNET BANK - BANK RAKYAT INDONESIA                 IDR   
4          FINNET BANK - BANK CENTRAL ASIA                 IDR   
...                                    ...                 ...   
95531                                    -                   -   
95532                                    -                   -   
95533                                    -                   -   
95534                                    -                   -   
95535                                    -                   -   

      billing_11_isInstallment  billing_11_paymentDate  billing_11_paymentDay  \
0                Bukan Cicilan                20201217                     17   
1                Bukan Cicilan                20201217                     17   
2                Bukan Cicilan                20201208                      8   
3                Bukan Cicilan                20201221                     21   
4                Bukan Cicilan                20201203                      3   
...                        ...                     ...                    ...   
95531                        -                       0                      0   
95532                        -                       0                      0   
95533                        -                       0                      0   
95534                        -                       0                      0   
95535                        -                       0                      0   

       billing_11_paymentTime  billing_11_period  billing_11_period_index  \
0                    164344.0           202012.0                     11.0   
1                    141735.0           202012.0                     11.0   
2                    200459.0           202012.0                     11.0   
3                     65812.0           202012.0                     11.0   
4                    160239.0           202012.0                     11.0   
...                       ...                ...                      ...   
95531                     0.0                0.0                      0.0   
95532                     0.0                0.0                      0.0   
95533                     0.0                0.0                      0.0   
95534                     0.0                0.0                      0.0   
95535                     0.0                0.0                      0.0   

      billing_11_status  billing_10_amountTotal  \
0                  PAID                341000.0   
1                  PAID                324500.0   
2                  PAID                146355.0   
3                  PAID                209660.0   
4                  PAID                275000.0   
...                 ...                     ...   
95531                 

In [ ]:
df['billing_3_paymentDay'].value_counts().sort_index()

0       428
1      1817
2      1456
3      1553
4      1381
5      1228
6      1302
7      1062
8      1126
9      1954
10     2546
11     1240
12     1265
13     1332
14     1178
15     1029
16     1788
17     1318
18     2608
19     5446
20     9375
21    10379
22    13011
23     7772
24     4948
25     4338
26     3229
27     2608
28     1828
29     1115
30     1815
31     2061
Name: billing_3_paymentDay, dtype: int64

# modeling

In [ ]:
sample_df = df.groupby('month_1_late').sample(n=3000, random_state=1)
sample_df

index  Unnamed: 0                       _id  billing_11_amountTotal  \
37022  37022       38145  617ff9a39da5e9001140dce9                300000.0   
95336  95336       99793  618053939da5e9001141cdba                     0.0   
71577  71577       75276  61802d629da5e90011416df5                331900.0   
88860  88860       93129  6180494d9da5e9001141b3b2                     0.0   
11757  11757       12149  617fd8da9da5e9001140775d                388250.0   
...      ...         ...                       ...                     ...   
32873  32873       33838  617ff47f9da5e9001140cc16                305500.0   
60528  60528       62657  618018be9da5e90011413ca9                311000.0   
27121  27121       27930  617fed1f9da5e9001140b502                355000.0   
10048  10048       10398  617fd6739da5e90011407086                414000.0   
26526  26526       27320  617fec489da5e9001140b2a0                302000.0   

                       billing_11_channel billing_11_currency  \
37022             FINNET SUBCA - ALFAMART                 IDR   
95336                                   -                   -   
71577             FINNET SUBCA - ALFAMART                 IDR   
88860                                   -                   -   
11757     FINNET BANK - BANK CENTRAL ASIA                 IDR   
...                                   ...                 ...   
32873  FINNET BANK - BANK SYARIAH MANDIRI                 IDR   
60528      FINNET BANK - PT POS INDONESIA                 IDR   
27121             FINNET SUBCA - INDOMART                 IDR   
10048               FPC - FINNET KOPEGTEL                 IDR   
26526     FINNET BANK - BANK CENTRAL ASIA                 IDR   

      billing_11_isInstallment  billing_11_paymentDate  billing_11_paymentDay  \
37022            Bukan Cicilan                20210126                     26   
95336                        -                       0                      0   
71577            Bukan Cicilan                20201221                     21   
88860                        -                       0                      0   
11757            Bukan Cicilan                20201224                     24   
...                        ...                     ...                    ...   
32873            Bukan Cicilan                20201202                      2   
60528            Bukan Cicilan                20201221                     21   
27121            Bukan Cicilan                20201221                     21   
10048            Bukan Cicilan                20210104                      4   
26526            Bukan Cicilan                20201224                     24   

       billing_11_paymentTime  billing_11_period  billing_11_period_index  \
37022                192906.0           202012.0                     11.0   
95336                     0.0                0.0                      0.0   
71577                195108.0           202012.0                     11.0   
88860                     0.0                0.0                      0.0   
11757                161845.0           202012.0                     11.0   
...                       ...                ...                      ...   
32873                172458.0           202012.0                     11.0   
60528                102327.0           202012.0                     11.0   
27121                 81759.0           202012.0                     11.0   
10048                125003.0           202012.0                     11.0   
26526                115440.0           202012.0                     11.0   

      billing_11_status  billing_10_amountTotal  \
37022              PAID                287500.0   
95336                 -                     0.0   
71577              PAID                343850.0   
88860                 -                     0.0   
11757              PAID                385250.0   
...                 ...                     ...   
32873              PAID           

In [ ]:
amount = ['billing_{}_amountTotal'.format(i) for i in range(2,12)]
channel = ['billing_{}_channel'.format(i) for i in range (2,12)]
payDay = ['billing_{}_paymentDay'.format(i) for i in range (2,12)]
late = ['month_{}_late'.format(i) for i in range (1,12)]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
col_used2 = amount + channel + payDay + late
col_used2.append('gladius.paketradius')
col_used2.append('usage')

In [ ]:
df_model = sample_df[col_used2].loc[df['month_1_late'] != 'UNPAID']
df_model.head(10)

billing_2_amountTotal  billing_3_amountTotal  billing_4_amountTotal  \
37022               275000.0               287500.0               287500.0   
95336               258500.0               258500.0               270250.0   
71577               347600.0                26219.0                30000.0   
88860               346500.0               362250.0               362250.0   
11757               368500.0               385250.0               385250.0   
80861               286000.0               312000.0               299000.0   
33470               308000.0               336000.0               322000.0   
33692               236500.0               247250.0               247250.0   
2814                 70070.0                75070.0                38048.0   
72704               346500.0               346500.0               362250.0   

       billing_5_amountTotal  billing_6_amountTotal  billing_7_amountTotal  \
37022               287500.0               275000.0               287500.0   
95336               258500.0               258500.0               165000.0   
71577               328900.0               358800.0               343850.0   
88860               362250.0               362250.0               346500.0   
11757               385250.0               385250.0               385250.0   
80861               299000.0               299000.0               286000.0   
33470               322000.0               322000.0               322000.0   
33692               247250.0               247250.0               247250.0   
2814                 36570.0                86570.0                31570.0   
72704               362250.0               346500.0               346500.0   

       billing_8_amountTotal  billing_9_amountTotal  billing_10_amountTotal  \
37022               287500.0                58225.0                287500.0   
95336               235000.0                    0.0                     0.0   
71577               343850.0               343850.0                343850.0   
88860               306233.0               315000.0                     0.0   
11757               385250.0               368500.0                385250.0   
80861               286000.0               463339.0                260000.0   
33470               308000.0               336000.0                322000.0   
33692               247250.0               247250.0                247250.0   
2814                 36570.0                31570.0                 36570.0   
72704               362250.0               346500.0                 27500.0   

       billing_11_amountTotal                    billing_2_channel  \
37022                300000.0              FINNET SUBCA - ALFAMART   
95336                     0.0              FINNET SUBCA - ALFAMART   
71577                331900.0              [4N2]PLASA TANTULAR SMG   
88860                     0.0  FINNET BANK - BANK NEGARA INDONESIA   
11757                388250.0                          SOPP9015PAP   
80861                     0.0             LOKET UBC AREA-2 JAKUT-1   
33470                325000.0                FPC - FINNET KOPEGTEL   
33692                247250.0              FINNET SUBCA - INDOMART   
2814                  37560.0  FINNET BANK - BANK NEGARA INDONESIA   
72704                536002.0                FPC - FINNET KOPEGTEL   

                         billing_3_channel  \
37022              FINNET SUBCA - ALFAMART   
95336              FINNET SUBCA - ALFAMART   
71577              FINNET SUBCA - INDOMART   
88860     FINNET SUBCA - MAGNA KARSA MULIA   
11757  FINNET BANK - BANK RAKYAT INDONESIA   
80861             FINNET SUBCA - TOKOPEDIA   
33470                FPC - FINNET KOPEGTEL   
33692                 FINNET WAY4 - FINNET   
2814        FINNET BANK - PT POS INDONESIA   
72704            FINNET BANK - BANK JATENG   

                         billing_4_channel  \
37022              FINNET SUBCA - ALFAMART   
95336              FINNET SUBCA - ALFAMART   
715

In [ ]:
#code categorical data
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

for column in channel:
  df_model[column] = label.fit_transform(df_model[column])
channel_class = list(label.classes_)

for column in late:
  df_model[column] = label.fit_transform(df_model[column])
late_class = list(label.classes_)

df_model['gladius.paketradius'] = label.fit_transform(df_model['gladius.paketradius'])
paket_class = list(label.classes_)

In [ ]:
df_model.head(10)

billing_2_amountTotal  billing_3_amountTotal  billing_4_amountTotal  \
37022               275000.0               287500.0               287500.0   
95336               258500.0               258500.0               270250.0   
71577               347600.0                26219.0                30000.0   
88860               346500.0               362250.0               362250.0   
11757               368500.0               385250.0               385250.0   
80861               286000.0               312000.0               299000.0   
33470               308000.0               336000.0               322000.0   
33692               236500.0               247250.0               247250.0   
2814                 70070.0                75070.0                38048.0   
72704               346500.0               346500.0               362250.0   

       billing_5_amountTotal  billing_6_amountTotal  billing_7_amountTotal  \
37022               287500.0               275000.0               287500.0   
95336               258500.0               258500.0               165000.0   
71577               328900.0               358800.0               343850.0   
88860               362250.0               362250.0               346500.0   
11757               385250.0               385250.0               385250.0   
80861               299000.0               299000.0               286000.0   
33470               322000.0               322000.0               322000.0   
33692               247250.0               247250.0               247250.0   
2814                 36570.0                86570.0                31570.0   
72704               362250.0               346500.0               346500.0   

       billing_8_amountTotal  billing_9_amountTotal  billing_10_amountTotal  \
37022               287500.0                58225.0                287500.0   
95336               235000.0                    0.0                     0.0   
71577               343850.0               343850.0                343850.0   
88860               306233.0               315000.0                     0.0   
11757               385250.0               368500.0                385250.0   
80861               286000.0               463339.0                260000.0   
33470               308000.0               336000.0                322000.0   
33692               247250.0               247250.0                247250.0   
2814                 36570.0                31570.0                 36570.0   
72704               362250.0               346500.0                 27500.0   

       billing_11_amountTotal  billing_2_channel  billing_3_channel  \
37022                300000.0                104                 63   
95336                     0.0                104                 63   
71577                331900.0                324                 68   
88860                     0.0                 84                 69   
11757                388250.0                283                 46   
80861                     0.0                171                 75   
33470                325000.0                120                 79   
33692                247250.0                109                 77   
2814                  37560.0                 84                 62   
72704                536002.0                120                 36   

       billing_4_channel  billing_5_channel  billing_6_channel  \
37022                 58                 50                 53   
95336                 58                 50                 53   
71577                 64                 55                164   
88860                 35                 50                 31   
11757                 40                 18                 36   
80861                 71                 62                 64   
33470                 75                 32                 55   
33692                 57                 64                 58   
2814                  35                 27

In [ ]:
channel_class

['-',
 'BJM BLN CATEL BATULICIN',
 'BJM PLI PLS PLEIHARI',
 'Counter Desk Kancatel Samulaki',
 'DEPOSIT INDIHOME',
 'DIV ENTREPRISE SERVICE MANDIRI - DIVISI ENTERPRISE',
 'EDC BNI LOKET TUNGGAKAN PLASA RKT',
 'EDC MANDIRI KALIBATA',
 'FINNET BANK - ANJAK BANK BCA',
 'FINNET BANK - ANJAK BANK MEGA',
 'FINNET BANK - ANJAK CITIBANK',
 'FINNET BANK - ANZ PANIN BANK',
 'FINNET BANK - BANK BTN',
 'FINNET BANK - BANK BUANA INDONESIA',
 'FINNET BANK - BANK BUKOPIN',
 'FINNET BANK - BANK CENTRAL ASIA',
 'FINNET BANK - BANK DANAMON',
 'FINNET BANK - BANK EKONOMI',
 'FINNET BANK - BANK GANESHA',
 'FINNET BANK - BANK INTERNASIONAL INDONESIA',
 'FINNET BANK - BANK JATENG',
 'FINNET BANK - BANK MANDIRI',
 'FINNET BANK - BANK MUAMALAT INDONESIA',
 'FINNET BANK - BANK NAGARI',
 'FINNET BANK - BANK NEGARA INDONESIA',
 'FINNET BANK - BANK NIAGA',
 'FINNET BANK - BANK NISP',
 'FINNET BANK - BANK PERMATA',
 'FINNET BANK - BANK RAKYAT INDONESIA',
 'FINNET BANK - BANK RIAU KEPRI',
 'FINNET BANK - BANK SULUT

In [ ]:
late_class

['LATE PAID', 'PAID', 'UNPAID']

In [ ]:
paket_class

['-',
 'INET U1M',
 'INET U2M',
 'INET U512K',
 'INET10Q050',
 'INET10Q10',
 'INET10Q100',
 'INET10Q20',
 'INET10Q50',
 'INETBHOME',
 'INETF100M',
 'INETF10M',
 'INETF20M',
 'INETF30M',
 'INETF40M',
 'INETF50M',
 'INETFL10M',
 'INETFL20M',
 'INETFN10M',
 'INETFN20M',
 'INETFN30M',
 'INETHYBRID',
 'INETLOY10',
 'INETLOY20',
 'INETLOY200',
 'INETLOY30',
 'INETLOY300',
 'INETLOY40',
 'INETLOY50',
 'INETMBQ50',
 'INETMBSP10',
 'INETNLOY10',
 'INETNLOY100',
 'INETNLOY20',
 'INETNLOY200',
 'INETNLOY30',
 'INETNLOY300',
 'INETNLOY40',
 'INETNLOY50',
 'INETNLOYL10',
 'INETNLOYL20',
 'INETR10M',
 'INETR1M',
 'INETR2M',
 'INETR3M',
 'INETR512K',
 'INETR5M',
 'INETSSQ100',
 'INETSSQ20']

In [ ]:
X = df_model.drop('month_1_late', axis=1)
Y = df_model['month_1_late']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
from pyts.classification import TimeSeriesForest
model = TimeSeriesForest(n_estimators = 2000, random_state=43)
model.fit(x_train, y_train)

TimeSeriesForest(n_estimators=2000, random_state=43)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
# evaluating the model
print('Training Accuracy :', model.score(x_train, y_train))
print('Testing Accuracy :', model.score(x_test, y_test))

# confusion matrix
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_pred)

plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

# classification report
print('\nClassification report:')
cr = classification_report(y_test, y_pred)
print(cr)

Training Accuracy : 1.0
Testing Accuracy : 0.8127777777777778



Classification report:
              precision    recall  f1-score   support

           0       0.82      0.81      0.82       921
           1       0.80      0.82      0.81       879

    accuracy                           0.81      1800
   macro avg       0.81      0.81      0.81      1800
weighted avg       0.81      0.81      0.81      1800



In [ ]:
df_model.head(10)

billing_2_amountTotal  billing_3_amountTotal  billing_4_amountTotal  \
37022               275000.0               287500.0               287500.0   
95336               258500.0               258500.0               270250.0   
71577               347600.0                26219.0                30000.0   
88860               346500.0               362250.0               362250.0   
11757               368500.0               385250.0               385250.0   
80861               286000.0               312000.0               299000.0   
33470               308000.0               336000.0               322000.0   
33692               236500.0               247250.0               247250.0   
2814                 70070.0                75070.0                38048.0   
72704               346500.0               346500.0               362250.0   

       billing_5_amountTotal  billing_6_amountTotal  billing_7_amountTotal  \
37022               287500.0               275000.0               287500.0   
95336               258500.0               258500.0               165000.0   
71577               328900.0               358800.0               343850.0   
88860               362250.0               362250.0               346500.0   
11757               385250.0               385250.0               385250.0   
80861               299000.0               299000.0               286000.0   
33470               322000.0               322000.0               322000.0   
33692               247250.0               247250.0               247250.0   
2814                 36570.0                86570.0                31570.0   
72704               362250.0               346500.0               346500.0   

       billing_8_amountTotal  billing_9_amountTotal  billing_10_amountTotal  \
37022               287500.0                58225.0                287500.0   
95336               235000.0                    0.0                     0.0   
71577               343850.0               343850.0                343850.0   
88860               306233.0               315000.0                     0.0   
11757               385250.0               368500.0                385250.0   
80861               286000.0               463339.0                260000.0   
33470               308000.0               336000.0                322000.0   
33692               247250.0               247250.0                247250.0   
2814                 36570.0                31570.0                 36570.0   
72704               362250.0               346500.0                 27500.0   

       billing_11_amountTotal  billing_2_channel  billing_3_channel  \
37022                300000.0                104                 63   
95336                     0.0                104                 63   
71577                331900.0                324                 68   
88860                     0.0                 84                 69   
11757                388250.0                283                 46   
80861                     0.0                171                 75   
33470                325000.0                120                 79   
33692                247250.0                109                 77   
2814                  37560.0                 84                 62   
72704                536002.0                120                 36   

       billing_4_channel  billing_5_channel  billing_6_channel  \
37022                 58                 50                 53   
95336                 58                 50                 53   
71577                 64                 55                164   
88860                 35                 50                 31   
11757                 40                 18                 36   
80861                 71                 62                 64   
33470                 75                 32                 55   
33692                 57                 64                 58   
2814                  35                 27

In [ ]:
model.feature_importances_

array([0.00426416, 0.00424859, 0.00423716, 0.00488909, 0.00916115,
       0.00936303, 0.01314751, 0.00855754, 0.0095923 , 0.00907122,
       0.00934569, 0.00972435, 0.00995753, 0.0093529 , 0.01201717,
       0.0058126 , 0.00607621, 0.0059118 , 0.00419331, 0.00445065,
       0.00466127, 0.01666609, 0.01056165, 0.05409316, 0.00805262,
       0.00804078, 0.008043  , 0.00526215, 0.00896796, 0.00902447,
       0.00099137, 0.        , 0.        , 0.00841835, 0.00898833,
       0.00917707, 0.0065467 , 0.00984638, 0.00809603, 0.0044078 ,
       0.00457989, 0.00475345, 0.00323204, 0.00206299, 0.00316826,
       0.00842578, 0.00911882, 0.00946103, 0.00422199, 0.0046228 ,
       0.00512351, 0.00267224, 0.001971  , 0.00339624, 0.00426186,
       0.00431113, 0.00413514, 0.0024766 , 0.00196145, 0.0035619 ,
       0.01075672, 0.02086234, 0.01642443, 0.00678634, 0.00971094,
       0.00884626, 0.00350657, 0.00108693, 0.00172435, 0.01539402,
       0.03026542, 0.02206706, 0.00530377, 0.00574649, 0.00566

In [ ]:
import pickle

pickle.dump(model, open('model_sample_2.p','wb'))

In [ ]:
pickle.dump(channel_class, open('channel_sample_2.p', 'wb'))
pickle.dump(late_class, open('late_sample_2.p', 'wb'))
pickle.dump(paket_class, open('paket_sample_2', 'wb'))

In [ ]:
pickle.dump(paket_class, open('paket_sample_2.p', 'wb'))

cek label encoder di setiap billing channel, misal kolom 1 indomaret kode 0, apakah indomaret di kolom 2 juga kode 0?
cek jumlah channel tiap kolomnya sama atau enggak

cara cek
df['billing_1_channel].unique
df['billing_2_channel].unique
